In [1]:
from detectors.dna.config import BlackBoxDnaConfig, NGramsConfigGenerator
from detectors.models.config import CompletionModelConfig
from detectors.dna.model import BlackBoxDNADetector
from pydantic import SecretStr
import os
import dotenv

dotenv.load_dotenv()

True

In [3]:
model_config = CompletionModelConfig(
    api_key=SecretStr(os.getenv("OPENAI_API_KEY")),
    model_name="gpt-4o-mini",
    user_prompt="Complete the following sentences for a total of around {n_words} words:",
    system_prompt="You are a helpful assistant that continues the sentences provided.",
    temperature=0.5
)

ngrams_config = NGramsConfigGenerator()
"""
N_min=4,
N_max=25,
stemmer=PorterStemmer(),
func=lambda n: n * math.log(n),
stopwords=spacy.load("en_core_web_sm").Defaults.stop_words
"""

blackbox_config = BlackBoxDnaConfig(
    truncation=0.5,
    K=10,
    threshold=0.00025,
    model_config=model_config,
    ngrams_config=ngrams_config,
)

In [6]:
detector = BlackBoxDNADetector(blackbox_config)

In [7]:
from datasets import load_dataset

In [8]:
xlsum = load_dataset('anakib1/mango-truth', 'xlsum')

In [9]:
xlsum = xlsum.remove_columns(["prompt", "user_id"])

In [10]:
from tqdm import tqdm
import random

y_true, y_pred = [], []

for _ in tqdm(range(100)):
    i = random.randint(0, len(xlsum["train"]) - 1)
    proba = detector.predict_proba(xlsum["train"][i]["output"])
    y_true.append(1 if xlsum["train"][i]["label"] else 0)
    y_pred.append(proba[1])

  3%|▎         | 3/100 [00:15<08:13,  5.08s/it]


KeyboardInterrupt: 

In [ ]:
from detectors.utils.training import calculate_classification
import numpy as np

calculate_classification(np.array(y_true), np.array(y_pred))

In [15]:
detector.predict_proba('Ми зредукували задачу SUBSET-SUM до задачі цілочисельного лінійного програмування (CLP), показавши, що будь-який екземпляр SUBSET-SUM може бути записаний у вигляді системи лінійних нерівностей. Це доводить, що CLP є щонайменше NP-складною, оскільки SUBSET-SUM відома як NP-повна задача.')

array([0., 1.])

In [14]:
detector.get_labels()

['Human', 'AI']